# Improving Jailbreak Robustness with LAT

This notebook uses LAT to greatly improve over refusal training's ability to make an LLM robust to jailbreaks.

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import torch
import sys
from dotenv import load_dotenv
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig

os.chdir("../")
cwd = os.getcwd()
if cwd not in sys.path:
    sys.path.insert(0, cwd)

from latent_at import *
from tasks.harmbench.HarmBenchTask import HarmBenchTask
from tasks.harmbench.FastHarmBenchEvals import run_attack_evals

load_dotenv()
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

## Configuration

Set whether to use Llama2-7B or Llama3-8B.

In [ ]:
use_llama2 = True
if use_llama2:  # use llama2-7b
    model_name = "meta-llama/Llama-2-7b-chat-hf"
else: # use llama3-8b
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

## Model

In [ ]:
model_dtype = torch.bfloat16
device = "cuda"
run_start_evals = False

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_access_token,
    torch_dtype=model_dtype
).to(device)


if "Llama-2" in model_name:
    model_type = "llama2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.padding_side = "left"
elif "Llama-3" in model_name:
    model_type = "llama3"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.padding_side = "left"
elif "zephyr" in model_name or "mistral" in model_name:
    model_type = "zephyr"    
    tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
    tokenizer.pad_token_id = tokenizer.unk_token_id
    tokenizer.padding_side = "left"
else:
    print(model_name)
    raise Exception("Unsupported model type?")

## Data

In [ ]:
advbench_data = HarmBenchTask(
    tokenizer=tokenizer,
    gen_batch_size=1,
    cls_batch_size=1,
    device=device,
    data_name="advbench",
    train_test_split=.8
)

harmbench_data = HarmBenchTask(
    tokenizer=tokenizer,
    gen_batch_size=1,
    cls_batch_size=1,
    device=device,
    data_name="harmbench_text",
    train_test_split=.8,
    func_categories=["standard", "contextual"]
)

# train_behaviors = advbench_data.train_behaviors + harmbench_data.train_behaviors

sys_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

if model_type == "llama2":
    use_tokenizer_template = True
    custom_prompt_template = None
    custom_completion_template = None
elif model_type == "llama3":
    use_tokenizer_template = False
    custom_prompt_template = "<|start_header_id|>user<|end_header_id|\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    custom_completion_template="{completion}"
else:
    sys_prompt=""
    use_tokenizer_template = False
    custom_prompt_template = "<|user|>\n{prompt}</s> \n <|assistant|>\n"
    custom_completion_template="{completion}"
 
lat_dataset = process_generic_chat_dataset(
    tokenizer,
    dataset="Baidicoot/augmented_advbench_v4",
    adv_column="rejected",
    def_column="chosen",
    split="train",
    use_tokenizer_template=use_tokenizer_template,
    system_prompt=sys_prompt,
    custom_prompt_template=custom_prompt_template,
    custom_completion_template=custom_completion_template
)

lat_dataloader = DataLoader(
    lat_dataset,
    batch_size=16,
    shuffle=True,
    drop_last=True,
    collate_fn=LatentAdversarialTrainingDataCollator(
        tokenizer.pad_token_id,
        truncate_length=2048
    )
)

# interleaving supervised finetuning with LAT stabilizes training
sft_dataset = process_generic_chat_dataset(
    tokenizer,
    dataset="abhayesian/ultrachat-first-response-long",
    adv_column="refusal",
    def_column="response",
    split="train",
    use_tokenizer_template=use_tokenizer_template,
    system_prompt=sys_prompt,
    custom_prompt_template=custom_prompt_template,
    custom_completion_template=custom_completion_template,
    add_eos_token=True
)

sft_dataloader = DataLoader(
    sft_dataset,
    batch_size=16,
    shuffle=True,
    drop_last=True,
    collate_fn=LatentAdversarialTrainingDataCollator(
        tokenizer.pad_token_id,
        truncate_length=2048
    )
)

## Trainer

In [ ]:
with torch.no_grad():
    if run_start_evals:
        pre_train_evals = run_attack_evals(
            model,
            model_type=model_type,
            verbose=False,
        )
        
peft_config = LoraConfig(
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj"],
)

model = get_peft_model(model, peft_config)

pgd_trainer = ProjectedGradLAT(
    model=model,  # model
    dataloader=lat_dataloader,  # dataloader for lat
    sft_dataloader=sft_dataloader,  # dataloader for supervised finetuning
    adv_loss_coefs={"toward": 0.5, "away": 0.5,},  # adversary's loss coefs
    def_loss_coefs={"sft_toward": 2, "sft_away": 0, "toward": 0.5, "away": 0.5,},  # model's loss coefs
    pgd_layers=["embedding", 8, 16, 24, 30],  # what layers to attack
    pgd_iterations_per_step=16,  # how many steps of projected gradient descent to do
    model_layers=list(range(0, model.config.num_hidden_layers)),  # model layers to train
    epsilon=6.0,  # attack l2 constraint
    inner_learning_rate=1e-3,  # adversary lr
    outer_learning_rate=2e-5,  # model lr
    model_iterations_per_step=4,  # how many times to train on each step
    num_steps=100,  # number of epochs
    max_batch_per_acc=4,  # max size of a minibatch
    only_train_lora=True,  # train using low rank adapters
    l2_regularization=0,  # coef for l2 weight regularization
    model_layers_module="model.layers",  # where the model layers are
    reinitialize_dev_optim=True  # whether to reinitialize optimizer every lat step
)

## Run!

In [ ]:
pgd_trainer.train(project_name="jailbreaks_test")
# pgd_trainer.model.save_pretrained("jailbreaks_test_save")